# Match Curve Direction

2021-09-24

This notebook outlines efforts to refine `match_curve_direction` function used to ensure consistent offset directions for lot line cutters. Here's what we're dealing with:

![](./assets/2021-09-24_curve-direction-testlot.png)

In [1]:
import os
import sys
import pickle

In [2]:
# add udtools library to path
home_dir = os.environ.get('HOME')
udtools_module_path = os.path.join(home_dir, 'git', 'ud-tools-core', 'udtools')

if udtools_module_path not in sys.path:
  sys.path.append(udtools_module_path)

In [14]:
from OCC.Core.gp import gp_Pnt, gp_Vec, gp_Dir
from OCC.Core.GeomAPI import geomapi_To2d
from OCC.Core.BRep import BRep_Tool
from OCC.Core.BRepAdaptor import BRepAdaptor_Curve, BRepAdaptor_CompCurve
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge, BRepBuilderAPI_MakeWire

from archtools.geometry.common import get_edges, get_vertices
from archtools.geometry.conversion import wire_to_curve
from archtools.geometry.edge import get_endpoints, edge_to_wire
from archtools.geometry.reference import world_xy
from archtools.geometry.wire import (
  is_ccw, 
  get_vertices as get_wire_vertices,
  to_curve_from_vertices
) 
from archtools.connectors.wkt import coords_to_curve, coords_to_edge

In [4]:
# load test geoms, saved from zoninglot notebook
# with open('./assets/20210921_edge850.pickle', 'rb') as f:
#   edge = pickle.load(f)

with open('./assets/20210921_edge105.pickle', 'rb') as f:
  edge = pickle.load(f)

with open('./assets/20210921_bounds2022850130.pickle', 'rb') as f:
  bounds = pickle.load(f)

In [9]:
# copied from build_yard_cutter
edge_aswire = edge_to_wire(edge)
edge_curve = to_curve_from_vertices(edge_aswire)
bounds_curve = to_curve_from_vertices(bounds)

Currently the algorighm works like so: first, gets first derivative at the middle parameter (0.5) of c1 (`edge_curve`), giving midpoint and tangent vector. It also gets start and endpoints.

For c2 (`bounds_curve`) it gets Knots, makes variables to track midpoint, tangent and tangent as direction, then loops over segments defined as spans between knots (i.e. from knot 1 to knot 2... note they are 1-indexed)...

In [15]:
c2_mid = gp_Pnt()
c2_tan = gp_Vec()
c2_dir = gp_Dir()

In [19]:
# because Knots are 1-indexed, this will fail:
# bounds_curve.Knot(0)
# but this won't: 
bounds_curve.Knot(5)

1.0

In [20]:
# LocalD1 won't throw the same error, though,
# if you try to feed it Knots 0 to 1 as a span:
bounds_curve.LocalD1(0.5, 0, 1, c2_mid, c2_tan)

In the current algorithm, the for loop is going from 0 to knotcount - 1. It should be from 1 to knotcount instead. This error means the algorithm will work and throw no errors but skip the last segment in each curve so it will fail to correctly orient the edge curve if it aligns with the last segment.

This seems to tentatively fix it, however now the offset direction is ultimately conditioned on the orientation of the bounding curve, need to verify this is CCW at some point.

See `archtools` e01d45e

